# Top Restaurantes do bairro Pinheiros/SP via Zomato

### Importação de pacotes:

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}
response = requests.get("https://www.zomato.com/pt/sao-paulo-sp/best-pinheiros-restaurants", headers=headers)
html = response.content

In [3]:
type(html)

bytes

In [4]:
html = html.decode('utf-8')

In [5]:
type(html)

str

In [6]:
def trata_html(input):
    return " ".join(input.split()).replace('> <', '><')

html = trata_html(html)
html

'<!DOCTYPE html><html lang="pt-pt" prefix="og: http://ogp.me/ns#" ><head><meta charset="utf-8" /><meta content="origin" name="referrer" /><meta http-equiv="X-UA-Compatible" content="IE=edge" /><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={licenseKey:"700b8cfba9",applicationID:"147205669"};window.NREUM||(NREUM={}),__nr_require=function(e,t,n){function r(n){if(!t[n]){var i=t[n]={exports:{}};e[n][0].call(i.exports,function(t){var i=e[n][1][t];return r(i||t)},i,i.exports)}return t[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(e,t,n){function r(){}function i(e,t,n){return function(){return o(e,[u.now()].concat(c(arguments)),t?null:this,n),t?void 0:this}}var o=e("handle"),a=e(7),c=e(8),f=e("ee").get("tracer"),u=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var d=["setPageViewName","setCustomAttribute","setErrorHandler","finished","addToTrace","inlineHit","addReleas

In [7]:
soup = BeautifulSoup(html, 'html.parser')
soup

<!DOCTYPE html>
<html lang="pt-pt" prefix="og: http://ogp.me/ns#"><head><meta charset="utf-8"/><meta content="origin" name="referrer"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={licenseKey:"700b8cfba9",applicationID:"147205669"};window.NREUM||(NREUM={}),__nr_require=function(e,t,n){function r(n){if(!t[n]){var i=t[n]={exports:{}};e[n][0].call(i.exports,function(t){var i=e[n][1][t];return r(i||t)},i,i.exports)}return t[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(e,t,n){function r(){}function i(e,t,n){return function(){return o(e,[u.now()].concat(c(arguments)),t?null:this,n),t?void 0:this}}var o=e("handle"),a=e(7),c=e(8),f=e("ee").get("tracer"),u=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var d=["setPageViewName","setCustomAttribute","setErrorHandler","finished","addToTrace","inlineHit","addRelease"],

In [8]:
type(soup)

bs4.BeautifulSoup

In [9]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="pt-pt" prefix="og: http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="origin" name="referrer"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <script type="text/javascript">
   (window.NREUM||(NREUM={})).loader_config={licenseKey:"700b8cfba9",applicationID:"147205669"};window.NREUM||(NREUM={}),__nr_require=function(e,t,n){function r(n){if(!t[n]){var i=t[n]={exports:{}};e[n][0].call(i.exports,function(t){var i=e[n][1][t];return r(i||t)},i,i.exports)}return t[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(e,t,n){function r(){}function i(e,t,n){return function(){return o(e,[u.now()].concat(c(arguments)),t?null:this,n),t?void 0:this}}var o=e("handle"),a=e(7),c=e(8),f=e("ee").get("tracer"),u=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var d=["setPageViewName","setCustomAttribute","setErrorHandler","finished","addToTrace","inlineH

In [10]:
restaurantes = soup.find('div', {'class': 'ui cards'})
restaurantes

<div class="ui cards" id="orig-search-list"><!-- START SHOWING SEARCH RESULTS --><div class="card search-snippet-card search-card"><div class="content"><div class="js-search-result-li even near status 1" data-position="1-1" data-res_id="6702159" data-trprovider=""><article class="search-result first"><div class="pos-relative clearfix"><div class="row"><div class="col-s-6 col-m-4"><div class="search_left_featured clearfix"><a class="feat-img lazy" data-original="https://b.zmtcdn.com/data/pictures/chains/8/6700428/a3161ec52de5ef5acb865eadaea31519_featured_v2.jpg?fit=around%7C200%3A200&amp;crop=200%3A200%3B%2A%2C%2A" href="https://www.zomato.com/pt/sao-paulo-sp/a-figueira-rubaiyat-jardim-paulista/info" style=" background-image:url(https://b.zmtcdn.com/images/placeholder_200.png);background-repeat: repeat;"></a></div></div><div class="col-s-16 col-m-12 pl0"><div class="row"><div class="col-s-12"><div class="res-snippet-small-establishment mt5" style="margin-bottom: 7px;"><a class="zdark tt

In [11]:
dados_restaurante = restaurantes.find('div', {'class':'col-s-16 col-m-12 pl0'})
dados_restaurante

<div class="col-s-16 col-m-12 pl0"><div class="row"><div class="col-s-12"><div class="res-snippet-small-establishment mt5" style="margin-bottom: 7px;"><a class="zdark ttupper fontsize6" href="https://www.zomato.com/pt/sao-paulo-sp/pinheiros-restaurants/refeições-de-luxo " title="Refeições de luxo em Pinheiros">Refeições de luxo</a></div><a class="result-title hover_feedback zred bold ln24 fontsize0" data-result-type="ResCard_Name" href="https://www.zomato.com/pt/sao-paulo-sp/a-figueira-rubaiyat-jardim-paulista" title="figueira rubaiyat, Jardim Paulista">A Figueira Rubaiyat </a><div class="single-rating flex"><div class="flex"><div class="star" data-icon="" style="color:#1C1C1C;"></div><div class="star" data-icon="" style="color:#1C1C1C;"></div><div class="star" data-icon="" style="color:#1C1C1C;"></div><div class="star" data-icon="" style="color:#1C1C1C;"></div><div class="star" data-icon="" style="color:#cfcfcf;"></div></div><span class="rating-value">4.4</span><span class="revie

In [12]:
tipo_restaurante = restaurantes.find('div', {'class':'search-page-text clearfix row'})
tipo_restaurante

<div class="search-page-text clearfix row"><div class="clearfix"><span class="col-s-5 col-m-4 ttupper fontsize5 grey-text">Cozinhas: </span><span class="col-s-11 col-m-12 nowrap pl0"><a href="https://www.zomato.com/pt/sao-paulo-sp/restaurants/bbq" title="Churrasco">Churrasco</a>, <a href="https://www.zomato.com/pt/sao-paulo-sp/restaurants/grill" title="Grelhados">Grelhados</a>, <a href="https://www.zomato.com/pt/sao-paulo-sp/restaurants/brazilian" title="Brasileira">Brasileira</a></span></div><div class="res-cost clearfix"><span class="col-s-5 col-m-4 ttupper fontsize5 grey-text">Custo para dois:</span><span class="col-s-11 col-m-12 pl0">R$ 300</span></div><div class="res-timing clearfix" title="12:30 to 24:00 (Mon-Thu),12:30 to 01:00 (Fri-Sat),Closed (Sun)"><span class="col-s-5 col-m-4 ttupper fontsize5 grey-text left">Horário:</span><div class="col-s-11 col-m-12 pl0 search-grid-right-text"> 12:30 a 24:00 (Seg-Qui),12:30 a 01:00... </div><div class="clear"></div></div><div class="res-

In [13]:
card = []
card = {}

In [14]:
card['restaurante'] = [dados_restaurante.find('a', {'class':'result-title hover_feedback zred bold ln24 fontsize0'}).getText()]
card['nota'] = [dados_restaurante.find('span', {'class':'rating-value'}).getText()]
card['numero_de_avaliacoes'] = [dados_restaurante.find('span', {'class':'review-count medium'}).getText()]
card['tipo'] = [dados_restaurante.find('a', {'class':'zdark ttupper fontsize6'}).getText()]
card['endereco'] = [dados_restaurante.find('div', {'class':'col-m-16 search-result-address grey-text nowrap ln22'}).getText()]
card['cozinha'] = [tipo_restaurante.find('span', {'class':'col-s-11 col-m-12 nowrap pl0'}).getText()]
card['custo_para_dois'] = [tipo_restaurante.find('span', {'class':'col-s-11 col-m-12 pl0'}).getText()]

In [15]:
card

{'restaurante': ['A Figueira Rubaiyat '],
 'nota': ['4.4'],
 'numero_de_avaliacoes': ['(51 Opiniões)'],
 'tipo': ['Refeições de luxo'],
 'endereco': [' Rua Haddock Lobo, 1738, Cerqueira César, Jardim Paulista, São Paulo 10000'],
 'cozinha': ['Churrasco, Grelhados, Brasileira'],
 'custo_para_dois': ['R$ 300']}

In [16]:
restaurantes_pinheiros = pd.DataFrame(card, index=None)
restaurantes_pinheiros

,restaurante,nota,numero_de_avaliacoes,tipo,endereco,cozinha,custo_para_dois
0,A Figueira Rubaiyat,4.4,(51 Opiniões),Refeições de luxo,"Rua Haddock Lobo, 1738, Cerqueira César, Jard...","Churrasco, Grelhados, Brasileira",R$ 300
